In [ ]:
import qiime2
import pandas as pd
import numpy as np

Read metadata about the samples

In [ ]:
metadata = pd.read_table('../data/soil/88soils_modified_metadata.txt', index_col=0)

In [ ]:
metadata["ph"]

We need only pH

In [ ]:
y = metadata["ph"].values
y.shape

In [ ]:
ph = metadata["ph"].to_csv('ph.csv', index=True)

Import count data

In [ ]:
%%bash
qiime tools import \
    --input-path 238_otu_table.biom \
    --output-path 88soils.biom.qza \
    --type FeatureTable[Frequency]

qiime tools import \
    --input-path 88soils_taxonomy.txt \
    --output-path 88soils_taxonomy.qza \
    --type FeatureData[Taxonomy]

In [ ]:
# Load the table
table_art = qiime2.Artifact.load('88soils.biom.qza')
all_samples = table_art.view(pd.DataFrame)

In [ ]:
all_samples.head()

Select only those OTUs occuring at least 100 times

In [ ]:
%%bash
qiime feature-table filter-features \
    --i-table 88soils.biom.qza \
    --o-filtered-table 88soils_filt100.biom.qza \
    --p-min-frequency 100

In [ ]:
popular = qiime2.Artifact.load('88soils_filt100.biom.qza')
df_119 = popular.view(pd.DataFrame)
df_119

[Replace](https://docs.qiime2.org/2018.6/plugins/available/composition/add-pseudocount/) zeros with 1 in all samples

In [ ]:
%%bash

qiime composition add-pseudocount \
    --i-table 88soils_filt100.biom.qza \
    --p-pseudocount 1 \
    --o-composition-table 88soils_composition.biom.qza

Load the filtered and zero-replaced data

In [ ]:
table_1 = qiime2.Artifact.load('88soils_composition.biom.qza')
df = table_1.view(pd.DataFrame)

Compare with the data from Morton et al. paper

In [ ]:
xls = pd.read_excel('ph_exel.xlsx', engine='openpyxl')
xls.head()

In [ ]:
xls_otus = np.array(xls['#OTU_ID'])
xls_otus.shape

In [ ]:
org_otus = list(map(int, np.array(df.columns)))
len(org_otus)

Show OTUs which are has been filtered out by Morton et al

In [ ]:
diff = np.setdiff1d(org_otus, xls_otus)
diff = ["".join(item) for item in diff.astype(str)]
diff

Select the same OTUs in our dataframe

In [ ]:
final = df.loc[:, ~df.columns.isin(diff)]
final.shape

In [ ]:
final.to_csv('soil_116.csv', index=True)